In [2]:
from argparse import Namespace, ArgumentParser
import glob
import json
import ntpath
import os
import shutil
import sys
import xml.etree.ElementTree as ET

from tqdm import tqdm

対応している属性は以下に限ります。

images  
-file_name, height, width, id  
annotations  
-image_id, bbox, category_id, id, iscrowd, area  
categories  
-supercategory, name, id  

In [10]:
def load_voc_bbox(fn):
    info = Namespace()

    tree = ET.parse(fn)
    root = tree.getroot()

    # Image information
    info.image_fn = root.find("filename").text
    info.width = int(root.find("size").find("width").text)
    info.height = int(root.find("size").find("height").text)
    info.depth = int(root.find("size").find("depth").text)

    # Bounding boxes
    objects = []
    for obj in root.findall("object"):
        label = obj.find("name").text
        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)

        objects.append(Namespace(label=label, xmin=xmin, ymin=ymin, xmax=xmax, ymax=ymax))
    info.objects = objects
        
    return info

In [14]:
#"jpg", "png"のみ対応
def convert(annotation_dir,save_path):
    all_annotation_files = glob.glob(annotation_dir+"/*.xml")
    cat1id = {"silent":1, "low":2, "high":3}
    annotations = []
    images = []
    categories = []
    countimg=0
    countobj=0
    cat2id = {}
    
    for xml_fn in tqdm(all_annotation_files):
        countimg+=1
        info = load_voc_bbox(xml_fn)
        image_id = countimg
        image_base_fn = ntpath.basename(info.image_fn).replace("jp2","jpg")
        
        print(image_base_fn)
        images.append({
            "file_name": image_base_fn,
            "height": info.height,
            "width": info.height,
            "id": countimg
        })

        for obj in info.objects:
                countobj+=1
                #if obj.label not in cat1id:
                    #raise Exception("ID is not defined for {}".format(obj.label))
                cat_id = cat1id[obj.label]
                cat2id[countobj] = cat_id

                annotations.append({
                    "image_id": countimg,
                    "bbox": [obj.xmin, obj.ymin, obj.xmax - obj.xmin, obj.ymax - obj.ymin],
                    "category_id": cat_id, # Category ID is zero indexed!!
                    "id": countobj,
                    "iscrowd": 0,
                    "area": (obj.xmax - obj.xmin) * (obj.ymax - obj.ymin)
                })

    

    for index, cat_name in cat2id.items():
        categories.append({
            "supercategory": "cell",
            "name": cat_name,
            "id": index,
        })
    print(len(all_annotation_files))
    
    with open(save_path, 'w') as f:
        json.dump({"images": images, "annotations": annotations, "categories": categories}, f,indent=4)
        

In [15]:
dir='/Usersdata/kome/Drshirasaki/DrShirasaki/secretion_prediction/ssd/CytokineRelease_Dataset/CytokineRelease/Efficient_det_data/'
xml_tra_dir = dir + '/row_data/xml_tra20201012/'
xml_val_dir = dir + '/row_data/xml_val20201012/'
assert os.path.exists(xml_tra_dir)
assert os.path.exists(xml_val_dir)

save_dir = dir + '/datasets/datasets20201012/annotations/'
save_tra = save_dir+'/instances_train.json'
save_val = save_dir+'/instances_val.json'

os.makedirs(save_dir, exist_ok=True)

In [16]:
convert(xml_tra_dir, save_tra)
convert(xml_val_dir, save_val)

 17%|█▋        | 529/3191 [00:00<00:01, 2661.99it/s]

20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY331C1_0_rotate_45.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY295C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY035C1_2-color0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY083C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY226C1_3.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY054C1_2-color0-rotate_270.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY015C1_1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY199C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT6XY104C1_0_rotate_135.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY307C1_0_rotate_45.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY035C1_2-color0-rotate_270.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY335C1_1-color0-rotate_270.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT5XY011C1_2-color0-f1.jpg
20161028_Quad2_hILC2_day4_c

 31%|███       | 992/3191 [00:00<00:00, 2473.47it/s]

20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY334C1_1-color0-rotate_180.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY052C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY266C1_2-color0-f0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY080C1_3.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY026C1_1-color0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY139C1_1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY338C1_1_rotate_195.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY348C1_1-color0-f1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT5XY338C1_1_rotate_225.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY144C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY099C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY348C1_1_rotate_15.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY338C1_1_rotate_315.jpg
20161028_Quad2_hILC2_day4_crop_for_endsiz

 46%|████▌     | 1452/3191 [00:00<00:00, 2380.59it/s]

...

20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY005C1_1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY071C1_1-color0-f1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY100C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT5XY104C1_0_rotate_285.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY307C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY086C1_2.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY314C1_2.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT6XY093C1_1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY182C1_1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT5XY080C1_3-color0-rotate_270.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY266C1_2_rotate_285.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY197C1_2.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY175C1_2-color0-f1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT6XY172C1_3.jpg
20

 66%|██████▋   | 909/1368 [00:00<00:00, 2253.68it/s]

20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY304C1_3.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT6XY035C1_2-color0-rotate_180.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY266C1_2-color0-f1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY038C1_1-color0-rotate_270.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY334C1_2-color0-f0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT5XY336C1_1_rotate_285.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY220C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY035C1_2-color0-rotate_90.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY084C1_1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY320C1_2-color0-rotate_180.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY226C1_2.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY334C1_1-color0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY213C1_1.jpg
20161028_Quad2_hILC2

100%|██████████| 1368/1368 [00:00<00:00, 2351.13it/s]


20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY049C1_0-color0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY149C1_2.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT5XY348C1_1_rotate_135.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT5XY307C1_0-color0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY085C1_2.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY338C1_1-color0-rotate_270.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY071C1_1_rotate_45.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT5XY334C1_2.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY327C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY335C1_0-color0-f1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY140C1_3.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY331C1_0-color0-f0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY335C1_1-color0-f0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_sel

In [51]:
print(save_dir)

/Usersdata/kome/Drshirasaki/DrShirasaki/secretion_prediction/ssd/CytokineRelease_Dataset/CytokineRelease/Efficient_det_data//datasets/datasets20201012/annotations/


In [9]:
files = glob.glob(xml_tra_dir+"/*.xml")
traimg = "/Usersdata/kome/Drshirasaki/DrShirasaki/secretion_prediction/ssd/CytokineRelease_Dataset/CytokineRelease/Efficient_det_data/datasets/datasets20201012/train/"
imgs = glob.glob(traimg+"/*.jpg")
imgpath=[]
for img in imgs:
    imgpath.append(os.path.basename(img))
for i in files:
    i = os.path.basename(i).replace(".xml",".jpg")
    if i not in imgpath:
        print(i)
    print(i)
for i in files

20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY331C1_0_rotate_45.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY295C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY035C1_2-color0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY083C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY226C1_3.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY054C1_2-color0-rotate_270.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY015C1_1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY199C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT6XY104C1_0_rotate_135.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY307C1_0_rotate_45.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY035C1_2-color0-rotate_270.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY335C1_1-color0-rotate_270.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT5XY011C1_2-color0-f1.jpg
20161028_Quad2_hILC2_day4_c

20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY337C1_2.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT6XY302C1_1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY351C1_0-color0-rotate_90.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY038C1_1-color0-f1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY038C1_1-color0-rotate_180.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY335C1_1-color0-rotate_180.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY071C1_1-color0-f0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY105C1_3.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY191C1_2.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT5XY191C1_1_rotate_315.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT6XY006C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT5XY071C1_1-color0-f0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT5XY198C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_

20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT6XY287C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY050C1_3.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT3XY203C1_3.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY137C1_2.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY171C1_2.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT4XY331C1_0-color0-rotate_90.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY158C1_0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY335C1_0-color0-f1.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY080C1_3-color0-rotate_180.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT5XY026C1_1-color0-f0.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT2XY173C1_2.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY253C1_3.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY294C1_2.jpg
20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT5XY338C1_1.jpg
2016102

In [8]:
print(imgpath[:10])

['20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY001C1_2.jpg', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY002C1_0-color0-f1.jpg', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY002C1_0-color0-rotate_180.jpg', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY002C1_0-color0-rotate_90.jpg', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY002C1_0.jpg', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY002C1_0_rotate_105.jpg', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY002C1_0_rotate_135.jpg', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY002C1_0_rotate_15.jpg', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY002C1_0_rotate_285.jpg', '20161028_Quad2_hILC2_day4_crop_for_endsize_selectedT1XY002C1_0_rotate_315.jpg']
